# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [44]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:
weather_df = pd.read_csv("../output_data/weather_df.csv")
weather_df = weather_df.drop("Unnamed: 0",axis=1)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lavrentiya,65.5833,-171.0000,43.48,98,100,6.85,RU,1662413537
1,Ganzhou,28.8184,115.4327,70.74,61,6,3.13,CN,1662413537
2,Khislavichi,54.1871,32.1582,39.65,93,24,7.18,RU,1662413538
3,Bugene,-1.5911,31.1403,62.92,79,40,3.09,TZ,1662413538
4,Upernavik,72.7868,-56.1549,41.65,85,100,31.45,GL,1662413539


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# HINT: be sure to handle NaN values
weather_df = weather_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]]
Humidity = weather_df["Humidity"]

In [47]:
#Create a Humidity Heatmap layer
fig = gmaps.figure(center = [0,0],zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
new_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) 
                                & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0),:]


# Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.
new_weather_df = new_weather_df.dropna()
new_weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,Steinbach am Taunus,50.1667,8.5667,71.02,55,0,1.14,DE,1662413352
159,Nsanje,-16.9200,35.2620,72.63,65,0,5.68,MW,1662413599
171,Kiryat Gat,31.6100,34.7642,77.68,78,0,2.37,IL,1662413540
183,Angoche,-16.2325,39.9086,71.02,83,0,6.64,MZ,1662413607
190,Delījān,33.9905,50.6838,77.36,12,0,4.34,IR,1662413609


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = new_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
32,Steinbach am Taunus,50.1667,8.5667,71.02,55,0,1.14,DE,1662413352,
159,Nsanje,-16.9200,35.2620,72.63,65,0,5.68,MW,1662413599,
171,Kiryat Gat,31.6100,34.7642,77.68,78,0,2.37,IL,1662413540,
183,Angoche,-16.2325,39.9086,71.02,83,0,6.64,MZ,1662413607,
190,Delījān,33.9905,50.6838,77.36,12,0,4.34,IR,1662413609,


In [50]:
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,
}
# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        # hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        # hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
32,Steinbach am Taunus,50.1667,8.5667,71.02,55,0,1.14,DE,1662413352,Vienna House MQ Kronberg im Taunus
159,Nsanje,-16.9200,35.2620,72.63,65,0,5.68,MW,1662413599,Shumba Lodge
171,Kiryat Gat,31.6100,34.7642,77.68,78,0,2.37,IL,1662413540,Desert Gat
183,Angoche,-16.2325,39.9086,71.02,83,0,6.64,MZ,1662413607,Hotel Chando Angoche
190,Delījān,33.9905,50.6838,77.36,12,0,4.34,IR,1662413609,Rezvan Hotel
241,Bāfq,31.6128,55.4107,78.26,13,0,5.39,IR,1662413629,Hotel
294,Matagami,49.7502,-77.6328,71.71,30,0,4.61,CA,1662413649,Motel Le Caribou
298,Kumul,42.8000,93.4500,73.92,17,0,6.93,CN,1662413650,Π Hotel
310,Salta,-24.7859,-65.4117,72.19,23,0,9.22,AR,1662413655,Sheraton Salta Hotel
311,Malinyi,-8.9333,36.1333,71.49,47,0,4.25,TZ,1662413656,


In [51]:
# Save Data to csv
hotel_df.to_csv("hotel_output.csv")


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [53]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))